In [2]:
import pandas as pd

# show more cols
pd.set_option('display.max_columns', 500)

In [3]:
data_dep = pd.read_csv("../data/delta_combined_flights_2018_2022_preprocessed_LGA_Dep.csv")
data_dep.to_csv("../data/LGA_Flights_Departure.csv", index=False)
data_dep.head()

,FlightDate,DepDelay,Distance,CRSDepTime,CRSElapsedTime,CRSArrTime,DayOfWeek,Year,Month,AverageDelayPerFlightNumber,AverageDelayPerDistanceGroup,AverageDelayPerAircraft,Dest_ALB,Dest_ATL,Dest_BNA,Dest_BOS,Dest_BUF,Dest_BZN,Dest_CHS,Dest_CLT,Dest_CMH,Dest_CVG,Dest_DCA,Dest_DEN,Dest_DFW,Dest_DTW,Dest_FLL,Dest_GSP,Dest_IAH,Dest_JAX,Dest_MCO,Dest_MHT,Dest_MIA,Dest_MSN,Dest_MSP,Dest_MSY,Dest_MYR,Dest_OMA,Dest_ORD,Dest_ORF,Dest_PBI,Dest_PWM,Dest_RDU,Dest_RIC,Dest_RSW,Dest_SDF,Dest_SRQ,Dest_STL,Dest_TPA,Dest_TVC
0,2018-01-01,-2.0,762.0,479.0,75.0,394,1,2018,1,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,-5.0,1076.0,1245.0,117.0,2,1,2018,1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,23.0,1076.0,1110.0,84.0,794,1,2018,1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,-6.0,1076.0,895.0,118.0,693,1,2018,1,0.0,11.5,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,-1.0,950.0,839.0,95.0,614,1,2018,1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# ARRIVAL DATA - WITHOUT LIVE DEPARTURE DATA

In [10]:
data_arr = pd.read_csv("../data/delta_combined_flights_2018_2019_averages_regression_ready_lga.csv")

# keep only the rows which arrive at LGA
# keep the rows for which LGA = -1 (arriving at LGA)
data_arr = data_arr[data_arr['LGA'] == -1]

# drop the LGA col
data_arr = data_arr.drop(columns=['LGA'])

# Formats
data_arr["CRSDepTime"] = data_arr["CRSDepTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
data_arr["CRSArrTime"] = data_arr["CRSArrTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
data_arr["CRSElapsedTime"] = data_arr["CRSElapsedTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
data_arr.head()

# save to csv
data_arr.to_csv("../data/LGA_Flights_Arrival_NoDep.csv", index=False)

# ARRIVAL DATA - WITH LIVE DEPARTURE DATA

In [9]:
# open the CSV delta_combined_flights_2018_2022
df = pd.read_csv("../data/delta_combined_flights_2018_2019_averages.csv")

df = df.fillna(0)
df = df.iloc[:, 1:]

# keep the rows for which Destination is LGA
df = df[df['Dest'] == 'LGA']

# drop the Dest column
df = df.drop(columns=['Dest'])

# correct the timestamps to minutes
df["DepTimeMinutes"] = df["DepTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
df["CRSDepTime"] = df["CRSDepTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
df["CRSArrTime"] = df["CRSArrTime"].apply(lambda x: int(x / 100) * 60 + x % 100)
df["CRSElapsedTime"] = df["CRSElapsedTime"].apply(lambda x: int(x / 100) * 60 + x % 100)

features = [
    "FlightDate",
    "DepDelay",
    "DepTimeMinutes",
    "TaxiOut",
    "ArrDelay",
    "Distance",
    "Origin",
    #"CRSDepTime",
    "CRSElapsedTime",
    "CRSArrTime",
    "DayOfWeek",
    "Year",
    "Month",
    #"Tail_Number",
    "AverageDelayPerFlightNumber",
    "AverageDelayPerDistanceGroup",
    "AverageDelayPerAircraft",
]

# split the dataset based on the FlightDate for october 2019
data_regression = df[features]
data_regression.head()

# transform categorical features into dummy variables
data_regression = pd.get_dummies(data_regression, columns=["Origin"], dtype="int")#, "Tail_Number"])

data_regression.to_csv("../data/LGA_Flights_Arrival_Dep.csv", index=False)

In [16]:
data_regression.head()

,FlightDate,DepTimeMinutes,TaxiOut,ArrDelay,Distance,Dest,CRSDepTime,CRSElapsedTime,CRSArrTime,DayOfWeek,Year,Month,AverageDelayPerFlightNumber,AverageDelayPerDistanceGroup,AverageDelayPerAircraft,Origin_AGS,Origin_ATL,Origin_BOS,Origin_BTV,Origin_BUF,Origin_BZN,Origin_CLE,Origin_CLT,Origin_CMH,Origin_CVG,Origin_DEN,Origin_DFW,Origin_DTW,Origin_FLL,Origin_GRR,Origin_IAH,Origin_IND,Origin_JAX,Origin_MCO,Origin_MIA,Origin_MSN,Origin_MSP,Origin_MSY,Origin_MYR,Origin_OMA,Origin_ORD,Origin_PBI,Origin_PWM,Origin_RIC,Origin_RSW,Origin_SDF,Origin_SRQ,Origin_STL,Origin_SYR,Origin_TPA,Origin_TVC
1,2018-01-01,516.0,14.0,-20.0,502.0,LGA,517.0,83.0,640,1,2018,1,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,2018-01-01,918.0,27.0,18.0,1076.0,LGA,926.0,143.0,1069,1,2018,1,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,2018-01-01,637.0,14.0,-25.0,950.0,LGA,640.0,111.0,791,1,2018,1,0.0,4.250000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,2018-01-01,986.0,13.0,36.0,950.0,LGA,970.0,116.0,1086,1,2018,1,0.0,1.888889,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
138,2018-01-01,795.0,12.0,-22.0,1096.0,LGA,795.0,140.0,975,1,2018,1,0.0,5.461538,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# save to csv
data_regression.to_csv("../data/delta_combined_flights_2018_2019_averages_regression_ready_lga.csv", index=False)

2019    27910
2018    11296
Name: Year, dtype: int64